In [1]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence

import sympy as sp
import numpy as np

import math

In [2]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init, order, r = get_processed_and_shifted_recurrence(laplace2d)

In [3]:
def scale_recurrence(r):
    #We want to subsitute s(i) r^i_{ct} = g(i)
    g = sp.Function("g")
    s = sp.Function("s")
    n = sp.symbols("n")
    rct = sp.symbols("r_{ct}")

    r_new = r*rct**n
    for i in range(order):
        r_new = r_new.subs(s(n-i),g(n-i)/(rct**(n-i)))

    return r_new

In [4]:
r_new = scale_recurrence(r)

In [5]:
max_abs = .0000001
var = _make_sympy_vec("x", 2)
rct = sp.symbols("r_{ct}")
g = sp.Function("g")
n = sp.symbols("n")
coord_dict = {var[0]: 1, var[1]: 1}

In [6]:
r_new

(-1)**(n + 1)*r_{ct}**n*((-1)**(n - 3)*r_{ct}**(3 - n)*(n + (n - 2)**3 - 2*(n - 2)**2 - 2)*g(n - 3)/(x0**3 + x0*x1**2) + (-1)**(n - 2)*r_{ct}**(2 - n)*(-n + 3*(n - 2)**2 + 2)*g(n - 2)/(x0**2 + x1**2) + (-1)**(n - 1)*r_{ct}**(1 - n)*(3*x0**2*(n - 2) + x0**2 + x1**2*(n - 2) - x1**2)*g(n - 1)/(x0**3 + x0*x1**2))

In [7]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs
derivs = compute_derivatives(10)

In [8]:
def evaluate_recurrence(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        subs_dict[g(i)] = get_recurrence(recur.subs(rct, rct_val), i).subs(subs_dict).subs(coord_dict)
        print(get_recurrence(recur.subs(rct, 1),i))
    return np.array(list(subs_dict.values()))

In [9]:
def evaluate_recurrence_lamb(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(-2)] = 0
    subs_dict[g(-1)] = 0
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        exp = get_recurrence(recur.subs(rct, rct_val), i)
        f = sp.lambdify([var[0], var[1], g(i-1), g(i-2), g(i-3)], exp)
        subs_dict[g(i)] = f(coord_dict[var[0]], coord_dict[var[1]], subs_dict[g(i-1)],
                            subs_dict[g(i-2)], subs_dict[g(i-3)])
    subs_dict.pop(g(-2))
    subs_dict.pop(g(-1))
    return np.array(list(subs_dict.values()))

In [10]:
def evaluate_true(coord_dict, rct_val, p):
    retMe = []
    for i in range(p):
        exp = (derivs[i]*rct_val**i)
        f = sp.lambdify(var, exp)
        retMe.append(f(coord_dict[var[0]], coord_dict[var[1]]))
    return np.array(retMe)

In [11]:
def compute_error(pw):
    x_coord = 10**(-pw)
    y_coord = 1
    var = _make_sympy_vec("x", 2)
    coord_dict = {var[0]: x_coord, var[1]: y_coord}

    rct_val = 1
    exp = evaluate_recurrence_lamb(coord_dict, rct_val, r_new, 9)
    true = evaluate_true(coord_dict, rct_val, 9)

    print(true)

    return np.abs(exp-true)/np.abs(true)

In [12]:
compute_error(8)

[ 0.00e+00 -1.00e-08  1.00e+00  6.00e-08 -6.00e+00 -1.20e-06  1.20e+02
  5.04e-05 -5.04e+03]


array([nan, 0, 1.11022302462516e-16, 2.20581496680807e-16, 0,
       0.192092895507813, 0.576278686523440, 548836844308037.,
       2.74418422154019e+15], dtype=object)

# Avoiding Cat Cancellation: Attempt 1
The question is can we avoid catastrophic cancellation in the recurrence when $x_0 << 1$? Where $(x_0, y_0)$ is the location of the source?

If we formulate a recurrence for
$$
g(i, x_0, y_0) = \frac{d^i}{dx^i}|_{x = 0} G(x, y) r_{ct}^i
$$
we will inevitably get catastrophic cancellation when $x_0 << y_0$. Suppose we let $r_{ct} = x_0$ (we can scale up and down later with the true $r_{ct}$) and have
$$
g(n, x_0, y_0) = f_1(x_0, y_0, n-1) g(n-1, x_0, y_0) + f_2(x_0, y_0, n-2) g(n-2, x_0, y_0) + f_3(x_0, y_0, n-3) g(n-3, x_0, y_0)
$$
we could treat $g(n-1, x_0, y_0), g(n-2, x_0, y_0), g(n-3, x_0, y_0)$ as constants and taylor expand  $f_i(x_0, y_0, j)$ when $x_0  << y_0$. So instead we get for example:
$$
g(2) = -g(1) + \frac{4g(1)}{x_1^2} \frac{x_0^2}{2!} - \frac{48 g(1)}{x_1^4} \frac{x_0^4}{4!} 
$$
$$
g(3) = -\frac{4(g(1)-2g(2))}{x_1^2} \frac{x_0^2}{2!} - \frac{48 (g(1)-2g(2))}{x_1^4} \frac{x_0^4}{4!} 
$$
$$
g(4) = g(3) - \frac{4(g(1)-5g(2)+3g(3))}{x_1^2 }  \frac{x_0^2}{2!} - \frac{48(g(1)-5g(2)+3g(3))}{x_1^4} \frac{x_0^4}{4!} 
$$
$$
g(5) = 2g(4) + \frac{8(3g(2)-6g(3)+2g(4))}{x_1^2} \frac{x_0^2}{2!}
$$

In [13]:
def generate_specialized_formula(i, order):
    a = sp.cancel(r_new.subs(rct, var[0]).subs(n, i))
    res = 0
    for j in range(order+1):
        res += sp.simplify(sp.diff(a, var[0], j).subs(var[0], 0)) * var[0]**j/math.factorial(j)
    return res

In [14]:
def evaluate_specialized_formula(coord_dict, p, rct_val, order_approx):
    subs_dict = {}
    subs_dict[g(-2)] = 0
    subs_dict[g(-1)] = 0
    subs_dict[g(0)] = derivs[0].subs(coord_dict)
    subs_dict[g(1)] = derivs[1].subs(coord_dict) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        exp = generate_specialized_formula(i, order_approx)
        f = sp.lambdify([var[0], var[1], g(i-1), g(i-2), g(i-3)], exp)
        subs_dict[g(i)] = f(coord_dict[var[0]], coord_dict[var[1]], subs_dict[g(i-1)],
                            subs_dict[g(i-2)], subs_dict[g(i-3)])
    subs_dict.pop(g(-2))
    subs_dict.pop(g(-1))
    return np.array(list(subs_dict.values()))

In [15]:
def compute_error_using_specialized_formula(pw, order_approx):
    x_coord = 10**(-pw)
    y_coord = 1
    var = _make_sympy_vec("x", 2)
    coord_dict = {var[0]: x_coord, var[1]: y_coord}

    rct_val = x_coord
    exp = evaluate_specialized_formula(coord_dict, 9, rct_val, order_approx)
    true = evaluate_true(coord_dict, rct_val, 9)
    print(exp)
    print(true)
    return np.abs(exp-true)/np.abs(true)

In [16]:
compute_error_using_specialized_formula(5, 25)

[5.00000041357686e-11 -9.99999999900000e-11 9.99999999700000e-11
 5.99999999800000e-20 -5.99999999400000e-20 -1.19999928158156e-28
 1.20000215021531e-28 8.61598124858447e-34 4.30496662438318e-33]
[ 5.00000041e-11 -1.00000000e-10  1.00000000e-10  6.00000000e-20
 -5.99999999e-20 -1.20000000e-28  1.20000000e-28  5.03999999e-37
 -5.03999998e-37]


array([0, 0, 1.29246970750185e-16, 4.01235405214419e-16,
       2.00617702740955e-16, 5.97982031121250e-7, 1.79394609774367e-6,
       1708.52009105628, 8542.60047595449], dtype=object)

In [17]:
compute_error(5)

[ 5.00000041e-11 -1.00000000e-05  1.00000000e+00  6.00000000e-05
 -5.99999999e+00 -1.20000000e-03  1.20000000e+02  5.03999999e-02
 -5.03999998e+03]


array([0, 0, 1.11022302495822e-16, 0, 4.44089210294152e-16,
       1.45828085314177e-7, 4.37484255134502e-7, 416.651671175665,
       2083.25836091981], dtype=object)

# Avoiding Cat Cancellation Attempt 2

We have
$$
\text{ Given } g(0), g(1), rct = 1
$$
$$
g(1) = \frac{1}{2\pi} \frac{x_0}{x_0^2 + x_1^2}
$$
$$
g(2) = \frac{x_0^2 -x_1^2}{x_0^3 +x_0 x_1^2} g(1)
$$
$$
g(3) = \frac{4x_0 g(2)}{x_0^2 +  x_1^2}  - \frac{2 g (1)}{x_0^2 + x_1^2}
$$
$$
g(4) = \frac{(7 x_0^2 + x_1^2)g(3)}{x_0^3 + x_0x_1^2}  - \frac{10g(2)}{x_0^2 + x_1^2} + \frac{2g(1)}{x_0^3 + x_0 x_1^2}
$$

Rewriting as an odd-even recurrence we get:
$$
g(2) =  \frac{1}{2\pi} \frac{x_0^2 -x_1^2}{(x_0^2 + x_1^2)^2}   
$$
$$
g(3) =  \frac{6x_0^2 -2x_1^2}{(x_0^2 + x_1^2)^2} g(1)
$$
$$
g(4) = \frac{(7 x_0^2 + x_1^2)}{x_0^2 + x_1^2} \left(\frac{4 g(2)}{x_0^2 +  x_1^2}  - \frac{1 }{\pi(x_0^2 + x_1^2)^2} \right) - \frac{10g(2)}{x_0^2 + x_1^2} + \frac{1}{x_0^2 + x_1^2} \frac{1}{\pi} \frac{1}{x_0^2 + x_1^2}
$$
$$
g(4) = \frac{18x_0^2 - 6x_1^2}{(x_0^2 + x_1^2)^2} g(2) + \frac{-(7 x_0^2 + x_1^2) + 1}{\pi(x_0^2 + x_1^2)^2}
$$

In [19]:
'''
x_plot = [i for i in range(len(compute_error(0)))]
for i in range(1, 4):
    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))
plt.xlabel("order of derivative being computed")
plt.ylabel("absolute error")
plt.title("recurrence error vs order for different source-locations")
plt.legend(title='ratio of x_{coord_src}/y_{coord_src}')
plt.show()
'''

'\nx_plot = [i for i in range(len(compute_error(0)))]\nfor i in range(1, 4):\n    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))\nplt.xlabel("order of derivative being computed")\nplt.ylabel("absolute error")\nplt.title("recurrence error vs order for different source-locations")\nplt.legend(title=\'ratio of x_{coord_src}/y_{coord_src}\')\nplt.show()\n'